In [2]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
import string
from nltk.stem.snowball import EnglishStemmer
from nltk.corpus import stopwords
from nltk.stem import porter
from nltk.stem.util import suffix_replace, prefix_replace
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import SnowballStemmer
import unicodedata
import datetime
import seaborn as sns

from sklearn.decomposition import NMF as NMF_sklearn

In [12]:
final_df = pd.read_json('/Users/lesropro/the_dive/DATAthon/poller_opposites_election_hackathon/ignorefiles/dfs/sentiment_whole.json')

In [13]:
final_df

,Unnamed: 0,created_at,full_text,user,retweet_count,favorite_count,lang,retweeted_status,clean_text,description,...,hour,dayofweek,date,clean_loc,trump_mentioned_cnt,biden_mentioned_cnt,#_sent_trumpdisgrace,#_sent_creepyjoe,sent,sent_cat
0,0,2020-09-29 23:59:56,"Bide, a professional debater?? Hardly!! More ...","{'id': 117904481, 'id_str': '117904481', 'name...",0,0,en,None,bide a profession debat hard more like a profe...,Conservative/#MAGA/ #WWG1WGA /MbrNRA/#Trump202...,...,23,1,2020-09-29,None,0,0,0,0,-0.4627,-1
1,1,2020-09-29 23:59:58,RT @DarDar458: #PresidentialDebate #Debates202...,"{'id': 30140760, 'id_str': '30140760', 'name':...",1,0,und,{'created_at': 'Tue Sep 29 23:55:42 +0000 2020...,rt dardar458 presidentialdeb debates2020 trump...,Canadian 🇨🇦 Windmill Cancer Survivor. I follow...,...,23,1,2020-09-29,None,1,0,-1,0,0.0000,0
2,2,2020-09-30 00:00:00,How can Biden stand a chance against Trump in ...,"{'id': 281610530, 'id_str': '281610530', 'name...",25,100,en,None,how can biden stand a chanc against trump in t...,Airing Weekdays at 9AM ET on the Fox Business ...,...,0,2,2020-09-30,NY,3,1,0,0,0.7579,1
3,3,2020-09-30 00:00:02,RT @abbydphillip: Very reminiscent of pre-2018...,"{'id': 88223320, 'id_str': '88223320', 'name':...",351,0,en,{'created_at': 'Tue Sep 29 23:00:28 +0000 2020...,rt abbydphillip veri reminisc of pre2018 elect...,Politics reporter covering elections & voting ...,...,0,2,2020-09-30,NY,1,0,0,0,0.0000,0
4,4,2020-09-30 00:00:04,RT @RudyGiuliani: Why was Corrupt Biden’s son ...,"{'id': 44295160, 'id_str': '44295160', 'name':...",15672,0,en,{'created_at': 'Tue Sep 29 19:07:59 +0000 2020...,rt rudygiuliani whi was corrupt biden son be p...,"wife, mother, grandmother, southerner, conserv...",...,0,2,2020-09-30,FL,0,1,0,0,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140333,140332,2020-10-02 23:59:13,RT @realDonaldTrump: https://t.co/B4H105KVSs,"{'id': 27780458, 'id_str': '27780458', 'name':...",131058,0,und,{'created_at': 'Fri Oct 02 22:31:56 +0000 2020...,rt realdonaldtrump httpstcob4h105kvss,I’m just your average fed up & fired up Americ...,...,23,4,2020-10-02,WA,1,0,0,0,0.0000,0
140334,140333,2020-10-02 23:59:27,"@ScottAdamsSays Hi, my son died, please vote J...","{'id': 1081132982449119232, 'id_str': '1081132...",0,0,en,None,scottadamssay hi my son die pleas vote joe biden,Connecting with you\nThank you ^-^\n\n🌎🇫🇷🐸1️⃣7️⃣,...,23,4,2020-10-02,None,0,1,0,0,-0.3182,-1
140335,140334,2020-10-02 23:59:03,@FoxNews can stick it where the sun doesn't sh...,"{'id': 2290692901, 'id_str': '2290692901', 'na...",0,0,en,None,foxnew can stick it where the sun doesnt shine...,Making moves for change,...,23,4,2020-10-02,None,0,0,0,0,0.0000,0
140336,140335,2020-10-02 23:59:41,RT @aaronjcarpenter: I can’t stress this enoug...,"{'id': 1931084401, 'id_str': '1931084401', 'na...",27,0,en,{'created_at': 'Fri Oct 02 23:58:26 +0000 2020...,rt aaronjcarpent i cant stress this enough if ...,Retired Navy Pilot & proud Naval Academy gradu...,...,23,4,2020-10-02,NH,1,0,0,0,-0.5106,-1


In [10]:
content = final_df['clean_text']
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000, min_df=10)
X = vectorizer.fit_transform(content)
features = vectorizer.get_feature_names()

In [8]:
nmf = NMF_sklearn(n_components=25, max_iter=100, random_state=12345, alpha=0.0)
W = nmf.fit_transform(X)
H = nmf.components_

In [11]:
for i, row in enumerate(H):
        top_ten = np.argsort(row)[::-1][:10]
        print(np.array(features)[top_ten])

['process' 'begin' 'quarantin' 'immedi' 'covid19' 'recoveri' 'posit'
 'tonight' 'test' 'flotus']
['trump' 'support' 'covid' 'melania' 'fuck' 'say' 'coronavirus' 'campaign'
 'ladi' 'que']
['realdonaldtrump' 'thank' 'httpstcokifqoazeuw' 'httpstco58ssx7efuj'
 'liar' 'httpstcoranjvhaco' 'httpstcotroy9toc8x' 'httpstco73kpvwrugh'
 'httpstcozm6lqzgppq' 'httpstcorunkw7pxyv']
['thank' 'negat' 'hop' 'jill' 'concern' 'happi' 'messag' 'im' 'report'
 'everyon']
['did' 'stand' 'instruct' 'petebuttigieg' 'group' 'unit' 'just' 'state'
 'supremacist' 'presid']
['rt' 'debates2020' 'httpstcokifqoazeuw' 'year' 'today'
 'httpstco58ssx7efuj' 'projectlincoln' 'httpstcoranjvhaco' 'ntrump'
 'gtconway3d']
['proud' 'boy' 'stand' 'told' 'say' 'son' 'said' 'tell' 'black' 'hes']
['biden' 'joe' 'antifa' 'son' 'hunter' 'lie' 'idea' 'wear' 'campaign'
 'realjameswood']
['debat' 'won' 'presidenti' 'poll' 'compil' 'big' 'base' 'commiss' 'rule'
 'moder']
['want' 'shut' 'countri' 'open' 'joe' 'man' 'celebr' 'didnt' 'sleepi